In [13]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (112 kB/s)

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("L2").getOrCreate()

# Loading files into Spark DataFrame

In [15]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
cameras_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep="\t", header=True)
cameras_df = cameras_df.dropna()
cameras_df = cameras_df.drop_duplicates()
cameras_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42900361|R1006EGZ9CPJP5|B00M07I2YC|     645536663|Nikon COOLPIX AW1...|          Camera|          5|            1|          1|   N|                Y|As good as the re...|great camera, ver...| 2015-01-01|
|         US|   46413833| R1014HFB7LT6D|B005HY4TXS|     810992030|JOBY Micro Hybrid...|          Camera|          5|    

# Games which received votes greater than or equal to 20

In [16]:
cameras_subset_df = cameras_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
cameras_subset_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            1|          1|   N|                Y|
|          5|            5|          5|   N|                Y|
|          5|            1|          1|   N|                Y|
|          4|            0|          5|   N|                N|
|          5|            0|          0|   N|                N|
|          5|            1|          1|   N|                N|
|          1|            1|          1|   N|                Y|
|          5|            0|          0|   N|                N|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [17]:
# Filter Games with Votes greater than or equal to 20
total_votes_greater_than_20_percentage_df = cameras_subset_df.filter(cameras_subset_df["total_votes"] >= 20)
total_votes_greater_than_20_percentage_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          1|            1|         55|   N|                Y|
|          5|           24|         24|   N|                N|
|          5|           45|         51|   N|                Y|
|          5|           40|         44|   N|                Y|
|          1|           18|         21|   N|                Y|
|          5|           22|         25|   N|                Y|
|          4|          599|        610|   N|                N|
|          1|           63|         73|   N|                Y|
|          5|           27|         28|   N|                N|
|          5|           32|         37|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [18]:
# Filter Games which have 50% helpful vote percentage
helpful_votes_greater_than_50_percentage_df = total_votes_greater_than_20_percentage_df.filter(total_votes_greater_than_20_percentage_df["helpful_votes"]/
                                                                                               total_votes_greater_than_20_percentage_df["total_votes"] >= 0.5)
helpful_votes_greater_than_50_percentage_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           24|         24|   N|                N|
|          5|           45|         51|   N|                Y|
|          5|           40|         44|   N|                Y|
|          1|           18|         21|   N|                Y|
|          5|           22|         25|   N|                Y|
|          4|          599|        610|   N|                N|
|          1|           63|         73|   N|                Y|
|          5|           27|         28|   N|                N|
|          5|           32|         37|   N|                Y|
|          4|           24|         24|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



# Paid Helpful votes and Unpaid helpful votes

In [19]:
# Paid helpful reviews
paid_helpful_votes_greater_than_50_percentage_df = helpful_votes_greater_than_50_percentage_df.filter(helpful_votes_greater_than_50_percentage_df['vine']== 'Y')
paid_helpful_votes_greater_than_50_percentage_df.describe().show(10)

+-------+-----------------+------------------+------------------+----+-----------------+
|summary|      star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+-----------------+------------------+------------------+----+-----------------+
|  count|              607|               607|               607| 607|              607|
|   mean|4.098846787479407| 82.49093904448105| 88.79571663920923|null|             null|
| stddev|0.983417957496939|144.10161946676772|148.94170492770337|null|             null|
|    min|                1|               100|               100|   Y|                N|
|    max|                5|                99|                98|   Y|                Y|
+-------+-----------------+------------------+------------------+----+-----------------+



In [20]:
# Unpaid reviews
unpaid_helpful_votes_greater_than_50_percentage_df = helpful_votes_greater_than_50_percentage_df.filter(helpful_votes_greater_than_50_percentage_df['vine']== 'N')
unpaid_helpful_votes_greater_than_50_percentage_df.describe().show()

+-------+------------------+------------------+------------------+-----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes| vine|verified_purchase|
+-------+------------------+------------------+------------------+-----+-----------------+
|  count|             50516|             50516|             50516|50516|            50516|
|   mean| 3.861726977591258|60.542184654366935| 65.82423390608916| null|             null|
| stddev|1.4369122022574783|122.63668266856274|126.91235294703327| null|             null|
|    min|                 1|                10|               100|    N|                N|
|    max|                 5|               999|               999|    N|                Y|
+-------+------------------+------------------+------------------+-----+-----------------+



# Percentage of five-star reviews

In [21]:
number_of_paid_five_star_reviews = paid_helpful_votes_greater_than_50_percentage_df[paid_helpful_votes_greater_than_50_percentage_df['star_rating'] == 5].count()
total_number_of_paid_reviews = paid_helpful_votes_greater_than_50_percentage_df.count()
percentage_of_paid_five_star = round(float(number_of_paid_five_star_reviews) / float(total_number_of_paid_reviews),4)
print(f'Total Number of paid reviews {total_number_of_paid_reviews}')
print(f'Number of paid five star reviews {number_of_paid_five_star_reviews}')
print(f'Percentage of five star paid reviews {percentage_of_paid_five_star * 100}%')

Total Number of paid reviews 607
Number of paid five star reviews 257
Percentage of five star paid reviews 42.34%


# Percentage of five-star reviews

In [12]:
number_of_unpaid_five_star_reviews = unpaid_helpful_votes_greater_than_50_percentage_df[unpaid_helpful_votes_greater_than_50_percentage_df['star_rating']== 5].count()
total_number_of_unpaid_reviews = unpaid_helpful_votes_greater_than_50_percentage_df.count()
percentage_of_unpaid_five_star = round(float(number_of_unpaid_five_star_reviews) / float(total_number_of_unpaid_reviews),4)
print(f'Number of unpaid reviews {total_number_of_unpaid_reviews}')
print(f'Number of unpaid five star reviews {number_of_unpaid_five_star_reviews}')
print(f'Percentage of five star unpaid reviews {percentage_of_unpaid_five_star * 100}%')

Number of unpaid reviews 50516
Number of unpaid five star reviews 25216
Percentage of five star unpaid reviews 49.919999999999995%
